In [1]:
import glob
import json
import pandas as pd
from tqdm import tqdm

import os

# path = os.getcwd()
# os.chdir(f"{path}")

In [9]:
os.chdir(f"/Users/robinkong/Desktop/meta")

In [11]:
# data structure
# .
# ├── aps-dataset-metadata-2013
# │   └── PR
# │   └── PRA
# │   └── PRB
# │   ...
# │   └── RMP
# ├── json_to_csv.py
# ├── result_PR1.csv
# ├── result_PR2.csv
# ├── ...
# ├── result_RMP1.csv
# └── result.csv

import glob
import json
import pandas as pd
from tqdm import tqdm

def json_to_df(json_data): # json file to DataFrame in a particular format
    
    json_cols = json_data.keys()
    result = dict()
    
    if "id" in json_cols:
        result["doi"] = json_data["id"]
    else:
        result["doi"] = [""]
    
    if "authors" in json_cols:
        numAuthor = len(json_data["authors"])
        names = []
        surnames = []
        
        for author in json_data["authors"]:
            if "name" in author.keys():
                names.append(author["name"].lower().replace(" ", ""))
            else:
                names.append("")
            
            if "surname" in author.keys():
                surnames.append(author["surname"].lower())
        
        result["name"] = names
        result["order"] = list(range(1, numAuthor + 1))
        result["numAuthor"] = numAuthor
        
        # alphabetical order
        if numAuthor >= 4 and surnames == sorted(surnames):
            result["is_alpha"] = True
        else:
            result["is_alpha"] = False
    else:
        result["name"] = [""]
        result["order"] = [""]
        result["numAuthor"] = 0
        result["is_alpha"] = False
    
    if "date" in json_cols:
        result["date"] = json_data["date"]
    else:
        result["date"] = [""]
    
    if "articleType" in json_cols:
        result["articleType"] = json_data["articleType"]
    else:
        result["articleType"] = [""]
    
    if "journal" in json_cols:
        result["journal"] = json_data["journal"]["id"]
    else:
        result["journal"] = [""]
    
    for i in range(10):
        result["pacs" + str(i) + "0"] = 0
    
    if "classificationSchemes" in json_cols and "pacs" in json_data["classificationSchemes"].keys():
        for pac in json_data["classificationSchemes"]["pacs"]:
            if pac["id"][0] in list(map(str, range(10))):
                result["pacs" + pac["id"][0] + "0"] = 1
    
    return pd.DataFrame(result)

def init_df():
    
    cols = ["doi", "name", "order", "numAuthor", "is_alpha",
            "date", "articleType", "journal"] + ["pacs" + str(i) + "0" for i in range(10)]
    
    return pd.DataFrame(columns=cols)

# 'aps-dataset-metadata-2013' json files to each dataframe, then to .csv
for file_path in glob.glob("./aps-dataset-metadata-2013/*"):
    dir_name = file_path.split("/")[-1]
    result = init_df() # dataframe default
    
    for i, file_name in tqdm(enumerate(glob.glob(file_path + "/**", recursive=True), 1)):
        if file_name.endswith(".json"):        
            with open(file_name, "r") as j:
                json_data = json.loads(j.read())
                result = result.append(json_to_df(json_data), ignore_index=True)
                
        if i % 10000 == 0:
            file_id = dir_name + str(i // 10000)
            result.to_csv(f"./result_{file_id}.csv", index=False) # dataframe to csv
            result = init_df() # dataframe default
            
    # dataframe to csv (save)
    file_id = dir_name + str(i//10000 + 1)
    result.to_csv(f"./result_{file_id}.csv", index=False)

# csvs to one csv
result = init_df()

for file_name in tqdm(glob.glob("./*")):
    if file_name.endswith(".csv"):
        result = pd.concat([result, pd.read_csv(file_name)])

65259it [05:57, 182.46it/s]
1505it [00:02, 583.24it/s]
261it [00:00, 628.64it/s]
3225it [00:07, 449.19it/s]
48129it [03:10, 253.27it/s]
161346it [18:17, 146.99it/s]
46052it [04:15, 180.27it/s]
110192it [35:24, 51.86it/s] 
69570it [24:57, 46.45it/s] 
34532it [07:09, 80.36it/s] 
218it [00:00, 586.97it/s]
2012it [00:05, 338.11it/s]
100%|██████████| 63/63 [01:30<00:00,  1.43s/it]


In [19]:
result.to_csv(r"/Users/robinkong/Desktop/meta/raw_result.csv", index=False)

In [20]:
raw_result = pd.read_csv(r"/Users/robinkong/Desktop/meta/raw_result.csv")

print(raw_result.shape)
raw_result.head()

(3565235, 18)


,doi,name,order,numAuthor,is_alpha,date,articleType,journal,pacs00,pacs10,pacs20,pacs30,pacs40,pacs50,pacs60,pacs70,pacs80,pacs90
0,10.1103/PhysRevC.76.025802,c.ugalde,1.0,12,False,2007-08-09,article,PRC,0,0,1,0,0,0,0,0,0,0
1,10.1103/PhysRevC.76.025802,a.e.champagne,2.0,12,False,2007-08-09,article,PRC,0,0,1,0,0,0,0,0,0,0
2,10.1103/PhysRevC.76.025802,s.daigle,3.0,12,False,2007-08-09,article,PRC,0,0,1,0,0,0,0,0,0,0
3,10.1103/PhysRevC.76.025802,c.iliadis,4.0,12,False,2007-08-09,article,PRC,0,0,1,0,0,0,0,0,0,0
4,10.1103/PhysRevC.76.025802,r.longland,5.0,12,False,2007-08-09,article,PRC,0,0,1,0,0,0,0,0,0,0
